## **Web Scraping de TUCARRO.COM**

### **1. Importamos las respectivas librerías**

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

### **2. Hacemos el respectivo requerimiento a la página**

➡️ Necesitamos un indicador que nos diga si la conexion fue realmente efectiva, en este caso, **200** es el código que muestra una conexion exitosa.

🗒️ **Observación:** Para una mayor informacion, les dejo el siguiente link 🔗 https://developer.mozilla.org/es/docs/Web/HTTP/Status/200

In [2]:
html_text = requests.get('https://carros.tucarro.com.co/')
print(html_text.status_code)

200


➡️ Convertimos la pagina que vamos a scrapear en una variable para poder usarla luego como un registro de donde esta saliendo la informacion

In [3]:
conexion_url = html_text.url
print(conexion_url)

https://carros.tucarro.com.co/


### **3. Seleccionamos la grilla específica de donde vamos a sacar la información**

➡️ La informacion que necesitamos esta contenida en el siguiente elemento HTML *'ui-search-layout ui-search-layout--grid'*.

In [4]:
soup = BeautifulSoup(html_text.text, 'lxml')
cars_grill = soup.find('ol', class_ = 'ui-search-layout ui-search-layout--grid')
cars = cars_grill.find_all('li', class_ = 'ui-search-layout__item')

### **4. Extraemols la información correspondiente**

➡️ Necesitamos guardar la informacion que vamos a extraer de la página web. Para esta aplicacion, vamos a usar la memoria del sistema para almancenar los datos a traves de un dataframe.

🗒️ **Observación:** Una aplicación real requiere montar un back-end donde la información persista una vez es descargada

In [5]:
# Declaramos un datafram vacio en el cual vamos a guardar la informacion posteriormente
df = pd.DataFrame()

#### 4.1 Nombre del carro

➡️ Iniciamos a iterarar para buscar cada uno de los elementos que necesitamos encontrar.

In [6]:
#Creamos una lista vacio donde vamos a almacenar la informacion
car_name_list = []

for car in cars:
    car_name = car.find('img', alt = True)
    
    #Construimos las expresión regular para identificar el nombre del carro
    pattern = r'<img[^>]*alt="([^"]*)"'
    car_name_adjusted = re.findall(pattern, str(car_name))
    car_name_list.append(car_name_adjusted)

df = pd.DataFrame(car_name_list, columns=['Nombre_del_vehiculo'])
print(df.head(5))

                 Nombre_del_vehiculo
0                       Mg Zs 1.5 Zs
1        Mazda Bt-50 2.2 4x2 Gasagol
2                 Kia Rio 1.4 Zenith
3           Chevrolet Tracker 1.8 Ls
4  Suzuki Vitara 1.6 2wd At 4x2 Live


#### 4.2 Precio del carro

In [7]:
#Creamos una lista vacio donde vamos a almacenar la informacion
price_car_list = []

for car in cars:
    price_car = car.find('span', class_ = 'andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript')

    #Construimos las expresión regular para identificar el precio
    pattern = r'aria-label="(\d+)'

    price_car_adjusted = re.findall(pattern, str(price_car))
    price_car_adjusted = int(price_car_adjusted[0])
    price_car_list.append(price_car_adjusted)

df['Precio_del_vehiculo'] = price_car_list
print(df.head(5))

                 Nombre_del_vehiculo  Precio_del_vehiculo
0                       Mg Zs 1.5 Zs             84500000
1        Mazda Bt-50 2.2 4x2 Gasagol             57000000
2                 Kia Rio 1.4 Zenith             72000000
3           Chevrolet Tracker 1.8 Ls             45800000
4  Suzuki Vitara 1.6 2wd At 4x2 Live             64000000


#### 4.3 Model y Kilometraje del carro

In [8]:
#Creamos una lista vacio donde vamos a almacenar la informacion
model_car_list = []
km_car_list = []

for car in cars:
    # Extraemos el modelo del carro
    model_car = int(car.find_all('li', class_ = 'ui-search-card-attributes__attribute')[0].text)
    model_car_list.append(model_car)

    # Extraemos el kilometraje del carro
    km_car = car.find_all('li', class_ = 'ui-search-card-attributes__attribute')[1].text
    km_car = int(km_car.replace(" Km","").replace(".",""))
    km_car_list.append(km_car)

df['Modelo_del_vehiculo'] = model_car_list
df['Kilometraje_del_vehiculo'] = km_car_list
print(df.head(5))

                 Nombre_del_vehiculo  Precio_del_vehiculo  \
0                       Mg Zs 1.5 Zs             84500000   
1        Mazda Bt-50 2.2 4x2 Gasagol             57000000   
2                 Kia Rio 1.4 Zenith             72000000   
3           Chevrolet Tracker 1.8 Ls             45800000   
4  Suzuki Vitara 1.6 2wd At 4x2 Live             64000000   

   Modelo_del_vehiculo  Kilometraje_del_vehiculo  
0                 2023                      9450  
1                 2011                    216000  
2                 2022                     37000  
3                 2015                    112000  
4                 2019                     64000  


#### 4.4 Ubicación del carro

In [9]:
#Creamos una lista vacio donde vamos a almacenar la informacion
location_car_list = []

for car in cars:
    location_car = car.find('span', class_ = 'ui-search-item__group__element ui-search-item__location').text
    location_car_list.append(location_car)

df['Ubicacion_del_vehiculo'] = location_car_list
print(df.head(5))

                 Nombre_del_vehiculo  Precio_del_vehiculo  \
0                       Mg Zs 1.5 Zs             84500000   
1        Mazda Bt-50 2.2 4x2 Gasagol             57000000   
2                 Kia Rio 1.4 Zenith             72000000   
3           Chevrolet Tracker 1.8 Ls             45800000   
4  Suzuki Vitara 1.6 2wd At 4x2 Live             64000000   

   Modelo_del_vehiculo  Kilometraje_del_vehiculo  Ubicacion_del_vehiculo  
0                 2023                      9450   Kennedy - Bogotá D.C.  
1                 2011                    216000  Fontibón - Bogotá D.C.  
2                 2022                     37000   Kennedy - Bogotá D.C.  
3                 2015                    112000      Suba - Bogotá D.C.  
4                 2019                     64000  Engativa - Bogotá D.C.  


In [10]:
#Creamos una lista vacio donde vamos a almacenar la informacion
links_car_list = []

for car in cars:
    link_car = car.find('a', class_ = 'ui-search-link')['href']
    links_car_list.append(link_car)

df['link'] = links_car_list
print(df.head(5))

                 Nombre_del_vehiculo  Precio_del_vehiculo  \
0                       Mg Zs 1.5 Zs             84500000   
1        Mazda Bt-50 2.2 4x2 Gasagol             57000000   
2                 Kia Rio 1.4 Zenith             72000000   
3           Chevrolet Tracker 1.8 Ls             45800000   
4  Suzuki Vitara 1.6 2wd At 4x2 Live             64000000   

   Modelo_del_vehiculo  Kilometraje_del_vehiculo  Ubicacion_del_vehiculo  \
0                 2023                      9450   Kennedy - Bogotá D.C.   
1                 2011                    216000  Fontibón - Bogotá D.C.   
2                 2022                     37000   Kennedy - Bogotá D.C.   
3                 2015                    112000      Suba - Bogotá D.C.   
4                 2019                     64000  Engativa - Bogotá D.C.   

                                                link  
0  https://articulo.tucarro.com.co/MCO-1452121427...  
1  https://articulo.tucarro.com.co/MCO-1452107915...  
2  https://

#### 4.5 Marca del vehiculo

➡️ Aun no sabemos la marca del vehículo, necesitamos extraerla de la columna *'Nombre_del_vehiculo'*

In [11]:
df['Marca_del_vehiculo'] = df['Nombre_del_vehiculo'].str.split(" ").str[0]
print(df.head(5))

                 Nombre_del_vehiculo  Precio_del_vehiculo  \
0                       Mg Zs 1.5 Zs             84500000   
1        Mazda Bt-50 2.2 4x2 Gasagol             57000000   
2                 Kia Rio 1.4 Zenith             72000000   
3           Chevrolet Tracker 1.8 Ls             45800000   
4  Suzuki Vitara 1.6 2wd At 4x2 Live             64000000   

   Modelo_del_vehiculo  Kilometraje_del_vehiculo  Ubicacion_del_vehiculo  \
0                 2023                      9450   Kennedy - Bogotá D.C.   
1                 2011                    216000  Fontibón - Bogotá D.C.   
2                 2022                     37000   Kennedy - Bogotá D.C.   
3                 2015                    112000      Suba - Bogotá D.C.   
4                 2019                     64000  Engativa - Bogotá D.C.   

                                                link Marca_del_vehiculo  
0  https://articulo.tucarro.com.co/MCO-1452121427...                 Mg  
1  https://articulo.tucarro.

#### 4.6 Timestand de la operacion y metadata

➡️ Es importante que siempre haya un registro historico de las operaciones para efectos de trazabilidad. Otra metadata que podriamos agregar es la pagina de la que se esta extrayendo la informacion

In [12]:
df['Registro_pagina_principal'] = pd.Timestamp.now()
df['origen_general'] = conexion_url
df

,Nombre_del_vehiculo,Precio_del_vehiculo,Modelo_del_vehiculo,Kilometraje_del_vehiculo,Ubicacion_del_vehiculo,link,Marca_del_vehiculo,Registro_pagina_principal,origen_general
0,Mg Zs 1.5 Zs,84500000,2023,9450,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452121427...,Mg,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/
1,Mazda Bt-50 2.2 4x2 Gasagol,57000000,2011,216000,Fontibón - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452107915...,Mazda,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/
2,Kia Rio 1.4 Zenith,72000000,2022,37000,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452146083...,Kia,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/
3,Chevrolet Tracker 1.8 Ls,45800000,2015,112000,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452085309...,Chevrolet,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/
4,Suzuki Vitara 1.6 2wd At 4x2 Live,64000000,2019,64000,Engativa - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452094465...,Suzuki,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/
5,Chevrolet Traverse 3.6 Rs At,207000000,2023,6900,Chapinero - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452106931...,Chevrolet,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/
6,Toyota Burbuja 4.5 Land Cruiser Vx,54900000,1999,200800,Caucasia - Antioquia,https://articulo.tucarro.com.co/MCO-1452106331...,Toyota,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/
7,Chevrolet Aveo 1.6 Five,24000000,2008,110060,Fontibón - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452144757...,Chevrolet,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/
8,Subaru Xv 2.0i Limited Eyesigth,96990000,2018,31604,Medellín - Antioquia,https://articulo.tucarro.com.co/MCO-1452080607...,Subaru,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/
9,Toyota Fortuner 2.7 4x2 Mecanica Gasolina,85000000,2013,156260,Engativa - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2529396890...,Toyota,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/


🗒️ **Observación:** En este punto podemos usar las propiedades de tablas SQL para vincular la informacion general que se está extrayendo en este punto, con la que vamos a extraer posteriormente para cada uno de los items.

Para tal efecto, necesitamos tener una llave que una ambas tablas, y en este caso, la llave es el identficador de cada vehículo que aparece en la url y que inicia con MCO.

In [13]:
def extract_mco_code(url):
    import re
    pattern = r'MCO-\d+'
    match = re.search(pattern, url)
    if match:
        return match.group()
    else:
        return None

df['Key'] = df['link'].apply(extract_mco_code)
df

,Nombre_del_vehiculo,Precio_del_vehiculo,Modelo_del_vehiculo,Kilometraje_del_vehiculo,Ubicacion_del_vehiculo,link,Marca_del_vehiculo,Registro_pagina_principal,origen_general,Key
0,Mg Zs 1.5 Zs,84500000,2023,9450,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452121427...,Mg,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/,MCO-1452121427
1,Mazda Bt-50 2.2 4x2 Gasagol,57000000,2011,216000,Fontibón - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452107915...,Mazda,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/,MCO-1452107915
2,Kia Rio 1.4 Zenith,72000000,2022,37000,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452146083...,Kia,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/,MCO-1452146083
3,Chevrolet Tracker 1.8 Ls,45800000,2015,112000,Suba - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452085309...,Chevrolet,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/,MCO-1452085309
4,Suzuki Vitara 1.6 2wd At 4x2 Live,64000000,2019,64000,Engativa - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452094465...,Suzuki,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/,MCO-1452094465
5,Chevrolet Traverse 3.6 Rs At,207000000,2023,6900,Chapinero - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452106931...,Chevrolet,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/,MCO-1452106931
6,Toyota Burbuja 4.5 Land Cruiser Vx,54900000,1999,200800,Caucasia - Antioquia,https://articulo.tucarro.com.co/MCO-1452106331...,Toyota,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/,MCO-1452106331
7,Chevrolet Aveo 1.6 Five,24000000,2008,110060,Fontibón - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1452144757...,Chevrolet,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/,MCO-1452144757
8,Subaru Xv 2.0i Limited Eyesigth,96990000,2018,31604,Medellín - Antioquia,https://articulo.tucarro.com.co/MCO-1452080607...,Subaru,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/,MCO-1452080607
9,Toyota Fortuner 2.7 4x2 Mecanica Gasolina,85000000,2013,156260,Engativa - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2529396890...,Toyota,2024-07-13 14:19:16.628647,https://carros.tucarro.com.co/,MCO-2529396890


### **5. Identificamos las páginas que vamos a consultar**

➡️ Para este ejercicio vamos a consultar unicamente las primeras 10 paginas disponibles. Vamos primero a declarar un diccionario que nos ayude a documentar las paginas en las que nos vamos a mover, la inicial termina con sufijo **"/_NoIndex_True"**

In [14]:
paginas = {}

➡️ Podemos observar que cada pagina tiene 48 elementos, y que la segunda pagina arranca en *"49 - _Desde_49_NoIndex_True"*, la tercera en *97 - _Desde_97_NoIndex_True*, es decir con 48 elementos de diferencia... 🤯 **"hay está el patrón"**

In [15]:
for i in range(1,11):
    if i == 1:
        paginas[str(i)] = f'_NoIndex_True'
    else:
        paginas[str(i)] = f'_Desde_{48*(i-1)+1}_NoIndex_True'
    print(paginas)

{'1': '_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True', '6': '_Desde_241_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True', '6': '_Desde_241_NoIndex_True', '7': '_Desde_289_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True', '6': '_Desde_241_NoIndex_True